In [1]:
#Import
import requests
import pandas as pd


#Read data from github

#Archivos json
url="https://raw.githubusercontent.com/mayteet/ProyectoIndividual/main/datasets/netflix_titles.json"
f=requests.get(url)
data=f.json()
# Convertir el archivo de json en un DataFrame
netflix=pd.DataFrame(data)

#Archivos csv
amazon= pd.read_csv("https://raw.githubusercontent.com/mayteet/ProyectoIndividual/main/datasets/amazon_prime_titles.csv")
disney= pd.read_csv("https://raw.githubusercontent.com/mayteet/ProyectoIndividual/main/datasets/disney_plus_titles.csv")
hulu= pd.read_csv("https://raw.githubusercontent.com/mayteet/ProyectoIndividual/main/datasets/hulu_titles.csv") 

#Agregamos la columna de plataform y unimos las tablas
netflix["plataform"]= "netflix"
amazon["plataform"]="amazon"
disney["plataform"]="disney"
hulu["plataform"]="hulu"


#Unimos los dataframes
df=pd.concat([netflix, amazon, disney, hulu]) 

data_all=pd.DataFrame([])
Lista_wrong=["min", "Seasons","Season"]
df_analysis= df[["show_id","rating","plataform"]]

for elemento in Lista_wrong:
    
    # Filtrar el Dataframe por columna rating en el que se encuentre la palabra min, Seasons, Season
    df_analysis_filter= df_analysis.loc[df_analysis["rating"].str.contains(elemento, na=False)]
    data_all=pd.concat([data_all,df_analysis_filter])
    
data_all.rename(columns={'rating':"duration"}, inplace=True)


#Sacamos las variables del dataframe principal
df_analysis_2= df[["show_id","duration","plataform"]]
df_analysis_2=df_analysis_2.dropna(subset="duration")

#Juntamos las columnas
data_all=pd.concat([data_all, df_analysis_2])
data_all

#Limpieza de datos
data_all.rename(columns={'duration':"duration_2"}, inplace=True)



#Juntamos todo en la tabla principal
df_final= pd.merge(df,data_all, on=["show_id","plataform"],how="left")

df_final= df_final.drop_duplicates(subset=["title","type"])

#Separamos date added en día mes y año
df_final[["day","month","year_added"]]=df_final.date_added.str.split(expand =True)


#Separamos la columna de duration por su valor y su unidad de medida
df_final[["duration_3","um"]]=df_final.duration.str.split(expand =True)

#Convertimos las columnas numéricas en tados numericos
df_final["duration_3"]=pd.to_numeric(df_final["duration_3"], downcast="integer")
df_final["year_added"]=pd.to_numeric(df_final["year_added"], downcast="integer")

#Nos aseguramos que la columna um tenga los datos limpios
df_final["um"]=df_final["um"].str.replace("Seasons","Season")


#Quitamos los duplicados de la tabla
df_final=df_final.drop_duplicates(["title","type","plataform"])
df_final


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2,day,month,year_added,duration_3,um
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",netflix,90 min,September,"25,",2021.0,90.0,min
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",netflix,2 Seasons,September,"24,",2021.0,2.0,Season
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,netflix,1 Season,September,"24,",2021.0,1.0,Season
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",netflix,1 Season,September,"24,",2021.0,1.0,Season
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,netflix,2 Seasons,September,"24,",2021.0,2.0,Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23008,s3064,TV Show,Samurai Harem,NaN,NaN,Japan,NaN,2009,TV-14,1 Season,"Anime, Comedy, Drama",His skill with a sword is magnificent but his ...,hulu,1 Season,NaN,NaN,NaN,1.0,Season
23009,s3065,TV Show,Scream Queens,NaN,NaN,United States,NaN,2015,TV-14,2 Seasons,"Comedy, Horror, Mystery",SCREAM QUEENS is a new genre-bending comedy-ho...,hulu,2 Seasons,NaN,NaN,NaN,2.0,Season
23013,s3069,TV Show,Star Trek: The Original Series,NaN,NaN,United States,NaN,1966,TV-PG,3 Seasons,"Action, Adventure, Classics",The 23rd century adventures of Captain James T...,hulu,3 Seasons,NaN,NaN,NaN,3.0,Season
23016,s3072,TV Show,The Twilight Zone,NaN,NaN,United States,NaN,1959,TV-PG,5 Seasons,"Classics, Science Fiction, Thriller",Rod Serling's seminal anthology series focused...,hulu,5 Seasons,NaN,NaN,NaN,5.0,Season


In [127]:
#CONSULTA 1

df_consul_1= df_final[["plataform","release_year","um","duration_3"]]
df_consul_1=df_consul_1[(df_consul_1["um"]=="min")&(df_consul_1["release_year"]==2019)&(df_consul_1["plataform"]=="netflix")].groupby("release_year").max()
df_consul_1

,plataform,um,duration_3
release_year,,,
2019,netflix,min,209.0


In [137]:
df_final["year_added"]=pd.to_numeric(df_final["year_added"], downcast="integer")

In [138]:
df_final["year_added"].info()

<class 'pandas.core.series.Series'>
Int64Index: 22242 entries, 0 to 23017
Series name: year_added
Non-Null Count  Dtype  
--------------  -----  
13020 non-null  float64
dtypes: float64(1)
memory usage: 347.5 KB


In [117]:
#CONSULTA 2
df_consul_2= df_final[["plataform","title"]]
df_consul_2=df_consul_2.groupby(["plataform"]).count()
df_consul_2

,title
plataform,
amazon,9338
disney,1399
hulu,2698
netflix,8807


In [118]:
#CONSULTA 3
df_consul_3= df_final[["plataform","listed_in"]]
df_consul_3=df_consul_3[df_consul_3["listed_in"].str.contains("TV Shows", regex=False)].groupby("plataform").count()
list= set(df_consul_3["listed_in"].tolist())
df_consul_3


,listed_in
plataform,
amazon,255
netflix,1755


In [122]:
#CONSULTA 4
df_consul_4=df_final[["plataform","cast","year_added"]]
df_consul_4= df_consul_4.dropna(axis=0, how="all", subset=["cast"])
df_consul_4=df_consul_4[df_consul_4["cast"].str.contains("Nell Carter", regex=False)].groupby(["year_added","plataform"]).count()
list= set(df_consul_4["cast"].tolist())
df_consul_4

,,cast
year_added,plataform,
2019,netflix,1


In [150]:
#CONSULTA 4
df_consul_4=df_final[["plataform","cast","year_added"]]
df_consul_4= df_consul_4.dropna(axis=0, how="all", subset=["cast"])
df_consul_4=df_consul_4[df_consul_4["cast"].str.contains("Nell Carter", regex=False)]
df_consul_4= df_consul_4[df_consul_4["year_added"]==2019]
df_consul_4=df_consul_4["cast"].count()
df_consul_4

1

In [81]:
#CONSULTA 3 NO CORRECTA
df_consul_3= df_final[["plataform","listed_in"]]
df_consul_3[df_consul_3["listed_in"]=="TV Shows"].groupby("plataform").count()

,listed_in
plataform,
amazon,255
netflix,16


In [80]:
df_consul_3[df_consul_3["listed_in"].str.contains("TV Shows", regex=False)].groupby("plataform").count()


,listed_in
plataform,
amazon,255
netflix,1755


In [89]:
list= set(df_consul_3["listed_in"].tolist())
list


{'Action & Adventure, Children & Family Movies, Cult Movies',
 'Animation, Drama, Family',
 'Action, Adventure, Thriller',
 'Comedy, Docuseries, Family',
 'Drama, History, International',
 'Animation, Comedy, Crime',
 'Crime TV Shows, TV Action & Adventure, TV Thrillers',
 'Comedy, Music, Reality',
 'Documentaries, International Movies',
 'Classic Movies, Cult Movies, Dramas',
 'Action-Adventure, Comedy, Family',
 'Biographical, Drama, Historical',
 'Drama, Kids, Special Interest',
 'Classic Movies, Dramas, International Movies',
 'Romantic TV Shows, TV Dramas, Teen TV Shows',
 'Action, Adventure, Crime',
 'Action, Anime, Drama',
 'Documentary, Kids',
 'Adventure, Drama, Kids',
 'Black Stories, Reality',
 'Comedies, Independent Movies',
 'Game Shows, Music, Reality',
 'Drama, Fantasy, Mystery',
 'Animals & Nature, Animation, Family',
 'Animals & Nature, Family, Reality',
 'Children & Family Movies, Classic Movies',
 'Black Stories, Horror',
 'Drama, Horror, Latino',
 'Black Stories, Cl

In [ ]:
#Obtener lista de duplicados
df_duplicados=df_final[df_final[["title","type"]].duplicated()]
list_duplicados=df_duplicados["title"].unique()
list_duplicados

#Crear un dataframe donde almacenare la información de las peliculas de las cuales he completado la informacion
data_total=pd.DataFrame([])


for elemento in list_duplicados:
    
    #Filtrar dataframe y ordener por categoría de country para cada pelicula
    df_prueba_fill=df_final[df_final["title"]==elemento].sort_values("cast", ascending= True)
    df_prueba_fill["cast_2"]=df_prueba_fill["cast"].fillna(method="ffill")
    data_total= pd.concat([data_total,df_prueba_fill])

data_total


In [33]:
#Obtener lista de duplicados
df_duplicados=df_final[df_final[["title","type"]].duplicated()]
list_duplicados=df_duplicados["title"].unique()


#Crear un dataframe donde almacenare la información de las peliculas de las cuales he completado la informacion
data_total=pd.DataFrame([])

#Tener una lista de análisis más reducida
df_cast=df_final[["show_id","type", "plataform","title","cast"]]



for elemento in list_duplicados:
    
    #Filtrar dataframe y ordener por categoría de country para cada pelicula
    df_prueba_fill=df_cast[df_cast["title"]==elemento].sort_values("cast", ascending= True)
    df_prueba_fill["cast_2"]=df_prueba_fill["cast"].fillna(method="ffill")
    data_total= pd.concat([data_total,df_prueba_fill])


#Sacamos las variables del dataframe principal
df_cast_prin= df[["show_id","type","plataform","title","cast"]]
df_cast_prin=df_cast_prin.dropna(subset="cast")
df_cast_prin.rename(columns={"cast": "cast_2"}, inplace=True)

#Juntamos las columnas
data_total_cast=pd.concat([data_total, df_cast_prin])
data_total_cast



,show_id,type,plataform,title,cast_2
1,s2,TV Show,netflix,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban..."
2,s3,TV Show,netflix,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi..."
4,s5,TV Show,netflix,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K..."
5,s6,TV Show,netflix,Midnight Mass,"Kate Siegel, Zach Gilford, Hamish Linklater, H..."
6,s7,Movie,netflix,My Little Pony: A New Generation,"Vanessa Hudgens, Kimiko Glenn, James Marsden, ..."
...,...,...,...,...,...
1445,s1446,Movie,disney,X-Men Origins: Wolverine,"Hugh Jackman, Liev Schreiber, Danny Huston, wi..."
1446,s1447,Movie,disney,Night at the Museum: Battle of the Smithsonian,"Ben Stiller, Amy Adams, Owen Wilson, Hank Azar..."
1447,s1448,Movie,disney,Eddie the Eagle,"Tom Costello, Jo Hartley, Keith Allen, Dickon ..."
1448,s1449,Movie,disney,Bend It Like Beckham,"Parminder Nagra, Keira Knightley, Jonathan Rhy..."
